In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Define dataset paths
base_path = '/kaggle/input/cityscapes-image-pairs/cityscapes_data'  # Update this path based on your Kaggle dataset location
outer_train_path = os.path.join(base_path, 'train')
outer_val_path = os.path.join(base_path, 'val')
inner_train_path = os.path.join(base_path, 'cityscapes_data', 'train')
inner_val_path = os.path.join(base_path, 'cityscapes_data', 'val')

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_images_from_folder(folder_path, target_size=(150, 150)):
    images = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if os.path.isfile(img_path):
            img = load_and_preprocess_image(img_path, target_size)
            images.append(img)
    return images

def load_dataset(base_path, target_size=(150, 150)):
    # Load images from all relevant folders
    train_images = load_images_from_folder(os.path.join(base_path, 'train'), target_size)
    val_images = load_images_from_folder(os.path.join(base_path, 'val'), target_size)
    inner_train_images = load_images_from_folder(os.path.join(base_path, 'cityscapes_data', 'train'), target_size)
    inner_val_images = load_images_from_folder(os.path.join(base_path, 'cityscapes_data', 'val'), target_size)
    
    # Combine all images
    all_images = train_images + val_images + inner_train_images + inner_val_images
    return np.array(all_images)

# Load the dataset
images = load_dataset(base_path)

# Create dummy labels for illustration (update with actual labels)
num_classes = 4  # Assuming 4 car colors
labels = np.random.randint(0, 2, size=(len(images),))  # Dummy binary labels

# Convert labels to categorical (one-hot encoding) for color prediction
color_labels = to_categorical(np.random.randint(0, num_classes, size=(len(images),)), num_classes=num_classes)

# Create dummy car counts, male counts, and female counts
car_count_labels = np.random.randint(0, 10, size=(len(images), 1))
male_count_labels = np.random.randint(0, 10, size=(len(images), 1))
female_count_labels = np.random.randint(0, 10, size=(len(images), 1))

# Train-test split
train_images, test_images, train_color_labels, test_color_labels, train_car_count_labels, test_car_count_labels, train_male_count_labels, test_male_count_labels, train_female_count_labels, test_female_count_labels = train_test_split(
    images, color_labels, car_count_labels, male_count_labels, female_count_labels, test_size=0.2, random_state=42)


2024-08-12 08:42:24.261319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 08:42:24.261572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 08:42:24.445079: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Function to create the model
def create_model():
    input_layer = layers.Input(shape=(150, 150, 3))
    x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    
    # Output layers
    output_color = layers.Dense(num_classes, activation='softmax', name='color_output')(x) # Assuming 4 car colors
    output_car_count = layers.Dense(1, activation='linear', name='car_count_output')(x)
    output_male_count = layers.Dense(1, activation='linear', name='male_count_output')(x)
    output_female_count = layers.Dense(1, activation='linear', name='female_count_output')(x)

    model = models.Model(inputs=input_layer, outputs=[output_color, output_car_count, output_male_count, output_female_count])
    
    model.compile(optimizer='adam', 
                  loss={'color_output': 'categorical_crossentropy', 
                        'car_count_output': 'mean_squared_error',
                        'male_count_output': 'mean_squared_error',
                        'female_count_output': 'mean_squared_error'},
                  metrics={'color_output': 'accuracy', 
                           'car_count_output': 'mse',
                           'male_count_output': 'mse',
                           'female_count_output': 'mse'})
    return model

model = create_model()

# Train the model
history = model.fit(train_images, 
                    {'color_output': train_color_labels,
                     'car_count_output': train_car_count_labels,
                     'male_count_output': train_male_count_labels,
                     'female_count_output': train_female_count_labels},
                    epochs=10,
                    validation_data=(test_images, 
                                     {'color_output': test_color_labels,
                                      'car_count_output': test_car_count_labels,
                                      'male_count_output': test_male_count_labels,
                                      'female_count_output': test_female_count_labels}))


Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 171s 958ms/step - car_count_output_mse: 9.7630 - color_output_accuracy: 0.2513 - female_count_output_mse: 9.6529 - loss: 30.6371 - male_count_output_mse: 9.8206 - val_car_count_output_mse: 8.8686 - val_color_output_accuracy: 0.2604 - val_female_count_output_mse: 8.4343 - val_loss: 26.8950 - val_male_count_output_mse: 8.1969
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 150s 862ms/step - car_count_output_mse: 8.6000 - color_output_accuracy: 0.2501 - female_count_output_mse: 8.6191 - loss: 27.1184 - male_count_output_mse: 8.5047 - val_car_count_output_mse: 8.3546 - val_color_output_accuracy: 0.2619 - val_female_count_output_mse: 8.5008 - val_loss: 26.4897 - val_male_count_output_mse: 8.2465
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 201s 858ms/step - car_count_output_mse: 8.5390 - color_output_accuracy: 0.2602 - female_count_output_mse: 8.2382 - loss: 26.5557 - male_count_output_mse: 8.3824 - val_car_count_output_mse: 8.5095 - val_color_output_accuracy: 0.2

In [4]:
# Evaluate the model on the test set
evaluation = model.evaluate(test_images, 
                            {'color_output': test_color_labels,
                             'car_count_output': test_car_count_labels,
                             'male_count_output': test_male_count_labels,
                             'female_count_output': test_female_count_labels})
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy (Color Prediction): {evaluation[1]}")

# Save the final model as a .keras file (new standard format)
model.save('final_model.keras')

# Save the final model as an .h5 file (legacy format)
model.save('final_model.h5')


44/44 ━━━━━━━━━━━━━━━━━━━━ 11s 239ms/step - car_count_output_mse: 9.0815 - color_output_accuracy: 0.2458 - female_count_output_mse: 8.5570 - loss: 27.0689 - male_count_output_mse: 8.0225
Test Loss: 26.553619384765625
Test Accuracy (Color Prediction): 8.674946784973145
